<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/research/autogen_agent_hanoi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code.

`AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`.


## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
# %pip install flaml[autogen]==2.0.0rc3

## Set your API Endpoint

The [`config_list_openai_aoai`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_openai_aoai) function tries to create a list of configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.
The following code looks for key files in key_file_path. Change it into the relevant path. If you open this notebook in google colab, you can upload your files by click the file icon on the left panel and then choose "upload file" icon. Then change the key file path.




In [2]:
from flaml import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location="..",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for feedback when it receives a "TERMINATE" signal from the assistant agent.

In [3]:
from flaml.autogen import AssistantAgent, UserProxyAgent
llm_config = {
    "request_timeout": 600,
    "seed": 43,
    "config_list": config_list,
    "model": "gpt-4",  # make sure the endpoint you use supports the model
    "temperature": 0,
}
assistant = AssistantAgent(
    name="proposer",
    llm_config=llm_config,
    system_message="""Consider the following puzzle problem:

Problem description:
- There are three lists labeled A, B, and C.
- There are three numbers -- 0, 1, and 2 -- distributed among those three lists.
- You can move numbers from the end of one list to the end of another list.
Rule #1: You can only move a number if it is at the end of its current list.
Rule #2: You can only move a number to the end of a list if it is larger than the other numbers in that list.

For a given configuration:
1. find the last number in each list and output, e.g.:
list A: a = empty
list B: b = 1
list C: c = 2
2. for each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source. 

Example: 
    list A: a = empty
    list B: b = 0
    list C: c = 2
    (A,B) or (B,A)? a is empty and b is not, so B->A
    (A,C) or (C,A)? a is empty and c is not, so C->A
    (B,C) or (C,B)? c>b, the list with larger number must be source, so C->B.

Example:
    list A: a = empty
    list B: b = empty
    list C: c = 2
    (A,B) or (B,A)? a and b are both empty, so skip
    (A,C) or (C,A)? a is empty and c is not, so C->A
    (B,C) or (C,B)? b is empty and c is not, so C->B.

3. Output the state after each candidate move. e.g.,
    From the given configuration, the state after (B->A) is (remove b from A and append to B): ...
    From the given configuration, the state after (C->B) is (remove c from C and append to B): ...
""",
)
# create a UserProxyAgent instance named "user"
user = AssistantAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
    # is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith("TERMINATE."),
    # code_execution_config=False,
    # llm_config=llm_config,
    # system_message="""You are a critic. You identify mistakes. If there is no mistake, reply TERMINATE to end the conversation."""
    # system_message="""You verify the solution and ask the user to check the entire sequence in case a step is missing. If the solution is correct, say "TERMINATE" to end the conversation.""",
    # system_message="""You verify whether a move is valid:
    # 1. say the state before a move.
    # 2. say the move.
    # 3. repeat the rules.
    # 4. use the rules to check whether it is valid and whether the result after move is correct. Do not assume a step is valid until you verify it.""",
    # work_dir='hanoi',
)

In [4]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = []
B = [0, 2]
C = [1]
""",
)

user (to proposer):

Current configuration:
A = []
B = [0, 2]
C = [1]


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = empty
    list B: b = 2
    list C: c = 1

2. For each pair of lists:
    (A,B) or (B,A)? a is empty and b is not, so B->A
    (A,C) or (C,A)? a is empty and c is not, so C->A
    (B,C) or (C,B)? b>c, the list with larger number must be source, so B->C.

3. Output the state after each candidate move:
    From the given configuration, the state after (B->A) is (remove b from B and append to A): A = [2], B = [0], C = [1]
    From the given configuration, the state after (C->A) is (remove c from C and append to A): A = [1], B = [0, 2], C = []
    From the given configuration, the state after (B->C) is (remove b from B and append to C): A = [], B = [0], C = [1, 2]

--------------------------------------------------------------------------------


In [5]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = []
B = [0, 1]
C = [2]
""",
)

user (to proposer):

Current configuration:
A = []
B = [0, 1]
C = [2]


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = empty
    list B: b = 1
    list C: c = 2

2. For each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source:
    (A,B) or (B,A)? a is empty and b is not, so B->A
    (A,C) or (C,A)? c>a, the list with larger number must be source, so C->A
    (B,C) or (C,B)? c>b, the list with larger number must be source, so C->B.

3. Output the state after each candidate move:
    From the given configuration, the state after (B->A) is (remove b from B and append to A): A = [1], B = [0], C = [2]
    From the given configuration, the state after (C->A) is (remove c from C and append to A): A = [2], B = [0, 1], C = []
    From the given configuration, the state after (C->B) is (remove c from C and append to B): A = [], B = [0, 1, 2

In [6]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = [0]
B = [1]
C = [2]
""",
)

user (to proposer):

Current configuration:
A = [0]
B = [1]
C = [2]


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = 0
    list B: b = 1
    list C: c = 2

2. For each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source:
    (A,B) or (B,A)? b>a, the list with larger number must be source, so B->A
    (A,C) or (C,A)? c>a, the list with larger number must be source, so C->A
    (B,C) or (C,B)? c>b, the list with larger number must be source, so C->B.

3. Output the state after each candidate move:
    From the given configuration, the state after (B->A) is (remove b from B and append to A): A = [0, 1], B = [], C = [2]
    From the given configuration, the state after (C->A) is (remove c from C and append to A): A = [0, 2], B = [1], C = []
    From the given configuration, the state after (C->B) is (remove c from C and append to B): 

In [7]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = [0]
B = [1, 2]
C = []
""",
)

user (to proposer):

Current configuration:
A = [0]
B = [1, 2]
C = []


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = 0
    list B: b = 2
    list C: c = empty

2. For each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source:
    (A,B) or (B,A)? b>a, the list with larger number must be source, so B->A
    (A,C) or (C,A)? a is not empty and c is empty, so A->C
    (B,C) or (C,B)? b is not empty and c is empty, so B->C

3. Output the state after each candidate move:
    From the given configuration, the state after (B->A) is (remove 2 from B and append to A): A = [0, 2], B = [1], C = []
    From the given configuration, the state after (A->C) is (remove 0 from A and append to C): A = [], B = [1, 2], C = [0]
    From the given configuration, the state after (B->C) is (remove 2 from B and append to C): A = [0], B = [1], C = [2]

----

In [8]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = []
B = [0, 1, 2]
C = []
""",
)

user (to proposer):

Current configuration:
A = []
B = [0, 1, 2]
C = []


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = empty
    list B: b = 2
    list C: c = empty

2. For each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source:
    (A,B) or (B,A)? a is empty and b is not, so B->A
    (A,C) or (C,A)? a and c are both empty, so skip
    (B,C) or (C,B)? b is not empty and c is empty, so B->C

3. Output the state after each candidate move:
    From the given configuration, the state after (B->A) is (remove b from B and append to A): 
    A = [2]
    B = [0, 1]
    C = []
    
    From the given configuration, the state after (B->C) is (remove b from B and append to C): 
    A = []
    B = [0, 1]
    C = [2]

--------------------------------------------------------------------------------


In [9]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = []
B = [0, 1, 2]
C = [3]
""",
)

user (to proposer):

Current configuration:
A = []
B = [0, 1, 2]
C = [3]


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = empty
    list B: b = 2
    list C: c = 3

2. For each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source:
    (A,B) or (B,A)? a is empty and b is not, so B->A
    (A,C) or (C,A)? c>a, the list with larger number must be source, so C->A
    (B,C) or (C,B)? c>b, the list with larger number must be source, so C->B.

3. Output the state after each candidate move:
    From the given configuration, the state after (B->A) is (remove b from B and append to A): A = [2], B = [0, 1], C = [3]
    From the given configuration, the state after (C->A) is (remove c from C and append to A): A = [3], B = [0, 1, 2], C = []
    From the given configuration, the state after (C->B) is (remove c from C and append to B): A = [], B =

In [10]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = [1]
B = [0, 2]
C = [3]
""",
)

user (to proposer):

Current configuration:
A = [1]
B = [0, 2]
C = [3]


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = 1
    list B: b = 2
    list C: c = 3

2. For each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source:
    (A,B) or (B,A)? b>a, the list with larger number must be source, so B->A
    (A,C) or (C,A)? c>a, the list with larger number must be source, so C->A
    (B,C) or (C,B)? c>b, the list with larger number must be source, so C->B.

3. Output the state after each candidate move:
    From the given configuration, the state after (B->A) is (remove 2 from B and append to A): A = [1, 2], B = [0], C = [3]
    From the given configuration, the state after (C->A) is (remove 3 from C and append to A): A = [1, 3], B = [0, 2], C = []
    From the given configuration, the state after (C->B) is (remove 3 from C and append 

In [11]:
user.initiate_chat(
    assistant,
    message="""Current configuration:
A = [1, 3]
B = [0, 2]
C = []
""",
)

user (to proposer):

Current configuration:
A = [1, 3]
B = [0, 2]
C = []


--------------------------------------------------------------------------------
proposer (to user):

1. Find the last number in each list:
    list A: a = 3
    list B: b = 2
    list C: c = empty

2. For each pair of lists (A, B), (A, C), (B, C), find which list has the larger number and put it as the source:
    (A,B) or (B,A)? a>b, the list with larger number must be source, so A->B
    (A,C) or (C,A)? c is empty and a is not, so A->C
    (B,C) or (C,B)? c is empty and b is not, so B->C

3. Output the state after each candidate move:
    From the given configuration, the state after (A->B) is (remove a from A and append to B): A = [1], B = [0, 2, 3], C = []
    From the given configuration, the state after (A->C) is (remove a from A and append to C): A = [1], B = [0, 2], C = [3]
    From the given configuration, the state after (B->C) is (remove b from B and append to C): A = [1, 3], B = [0], C = [2]

------